In [3]:
import scipy.io as sio
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import pickle
import os
from sklearn.manifold import SpectralEmbedding
from sklearn.gaussian_process import GaussianProcessRegressor

## Step 0: Split the data into train/test

In [5]:
datapath_base = "/data/yutaro/IROS/"

In [6]:
datapath_big = "/data/yutaro/IROS/sim_data_full_v11_d4_m1.mat"
datapath_small = "/data/yutaro/IROS/sim_data_partial_v111_d4_m1.mat"

In [5]:
big = sio.loadmat(datapath_big)
print(big['D'].shape)
pd.DataFrame(big['D']).head()

(1616064, 10)


,0,1,2,3,4,5,6,7,8,9
0,0.426909,118.443377,16.000000,16.000000,0.0,0.0,0.418931,118.442513,16.030001,16.030001
1,0.418931,118.442513,16.030001,16.030001,1.0,1.0,0.404663,118.443087,16.030001,16.030001
2,0.404663,118.443087,16.030001,16.030001,1.0,1.0,0.396586,118.443348,16.060001,16.060001
3,0.396586,118.443348,16.060001,16.060001,1.0,1.0,0.391399,118.442141,16.060001,16.060001
4,0.391399,118.442141,16.060001,16.060001,1.0,1.0,0.384889,118.439220,16.090002,16.090002


In [6]:
X_train, X_test, y_train, y_test = train_test_split(big['D'][:,:6], big['D'][:,6:], test_size=0.1, random_state=42)

## Step 1: Normalize the training data

In [10]:
def compute_normalization_parameters(data):
    """
    Compute normalization parameters (min, max per feature)
    :param data: matrix with data organized by rows [num_samples x num_features]
    :return: min and max per feautre as row matrices of dimension [1 x num_variables]
    """
    min_param = np.min(data, axis=0)
    max_param = np.max(data, axis=0)
    return np.expand_dims(min_param, 0), np.expand_dims(max_param, 0)

In [11]:
def normalize_data_per_row(data, min_param, max_param):
    """
    Normalize a given matrix of data (samples must be organized per row)
    :param data: input data
    :param min_param: min (for each feature) for normalization
    :param max_param: max (for each feature) for normalization
    :return: normalized data, (data - min_param) / max_param - min_param
    """
    # sanity checks!
    assert len(data.shape) == 2, "Expected the input data to be a 2D matrix"
    assert data.shape[1] == min_param.shape[1], "Data - min_param size mismatch ({} vs {})".format(data.shape[1], min_param.shape[1])
    assert data.shape[1] == max_param.shape[1], "Data - max_param size mismatch ({} vs {})".format(data.shape[1], max_param.shape[1])

    # TODO. Complete. Replace the line below with code to whitten the data.
    normalized_data = np.divide(data - min_param, max_param - min_param)

    return normalized_data

In [10]:
min_param, max_param = compute_normalization_parameters(X_train)

In [11]:
min_param_y, max_param_y = compute_normalization_parameters(y_train)

In [12]:
X_train_normalized = normalize_data_per_row(X_train, min_param, max_param)
X_test_normalized = normalize_data_per_row(X_test, min_param, max_param)
y_train_normalized = normalize_data_per_row(y_train, min_param_y, max_param_y)
y_test_normalized = normalize_data_per_row(y_test, min_param_y, max_param_y)

In [13]:
print(np.min(X_test_normalized, axis=1))
print(np.max(X_test_normalized, axis=1))
print(np.min(y_test_normalized, axis=1))
print(np.max(y_test_normalized, axis=1))

[0.27291292 0.21652865 0.         ... 0.26044959 0.         0.32468289]
[1.         1.         0.74025489 ... 1.         0.76048428 1.        ]
[0.27309061 0.21714172 0.23158517 ... 0.2603582  0.29895523 0.32491984]
[0.76699674 0.81079277 0.73955364 ... 0.76015749 0.76083998 0.79194657]


## Step 2: Implement the transition model 

- Given the query point $(x, a)$, we want to find where $(x,a)$ will go next.
- The function $(x_{next}, a_{next}) = f((x,a))$ should be implemented as follows:
  - Step1: Find K1 (=1000) nearest points in training data. Let the closest pair to $(x,a)$ be $(x_h, a_h)$
  - Step2: Diffusin map is created based on these 1000 points, which yields a reduced dimensional data (m=3 is used in the original paper)
  - Step3: K2 (=100) closest points in the reduced dimensional data to the $(x_h, a_h)$ in the diffusion map is found
  - Step4: Perform GP regression on these 100 points.

In [47]:
def transition_model(x_query, X_train_normalized, y_train_normalized, k1=1000, m=3, k2=100):
    file_path = os.path.join(datapath_base, 'nbrs.pkl')
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            nbrs = pickle.load(f)
    else:
        nbrs = NearestNeighbors(n_neighbors=k1, algorithm='ball_tree').fit(X_train_normalized)
        print("This is going to take about 15 min...")
        with open('nbrs.pkl', 'wb') as f:
            pickle.dump(nbrs, f)
    
    # Step1: Find K1 (=1000) nearest points in training data. Let the closest pair to $(x,a)$ be $(x_h, a_h)$
    distances, indices = nbrs.kneighbors(x_query.reshape(1,-1))
    X_selected = X_train_normalized[indices.reshape(-1), :]
    y_selected = y_train_normalized[indices.reshape(-1), :]
    
    # Step2: Diffusin map is created based on these 1000 points, which yields a reduced dimensional data
    embedding = SpectralEmbedding(n_components=m) 
    X_spectral = embedding.fit_transform(X_selected) 
    # y_spectral = embedding.fit_transform(y_selected) 
    
    # Step3: K2 (=100) closest points in the reduced dimensional data to the $(x_h, a_h)$ in the diffusion map is found
    nbrs_diffusion = NearestNeighbors(n_neighbors=k2, algorithm='ball_tree').fit(X_spectral)
    distances_diffusion, indices_diffusion = nbrs_diffusion.kneighbors(X_spectral[0, :].reshape(1, -1))

    # Step4: Perform GP regression on these 100 points.
    X_GP_input = X_spectral[indices_diffusion.reshape(-1), :]
    y_GP_input = y_selected[indices_diffusion.reshape(-1), :]
    #print("X_GP_input.shape: {}".format(X_GP_input.shape))
    #print("y_GP_input.shape: {}".format(y_GP_input.shape))

    gpr = GaussianProcessRegressor(kernel=None,random_state=0).fit(X_GP_input, y_GP_input)
    y_pred = gpr.predict(X_spectral[0,:].reshape(1,m)) 
    return y_pred

In [30]:
print(X_test_normalized[0,:].reshape(1,-1))

[[0.4367823  0.76776211 0.27291292 0.28221233 0.5        1.        ]]


In [38]:
y_new = transition_model(X_test_normalized[0,:].reshape(1,-1), X_train_normalized, y_train_normalized) 
print("y_new.shape: {}".format(y_new.shape))

X_GP_input.shape: (100, 3)
y_GP_input.shape: (100, 4)
y_new.shape: (1, 4)


## Step 3: Validate that the small data is indeed different from the big data

In [ ]:
def calc_mse(X_test_normalized, X_train_normalized, y_train_normalized):
    mse = 0
    for i in range(X_test_normalized.shape[0]):
        y_new = transition_model(X_test_normalized[i,:].reshape(1,-1), X_train_normalized, y_train_normalized)
        #print(y_test_normalized[i, :].reshape(1,-1).shape)
        #print(y_new.shape)
        mse += np.mean(np.linalg.norm(y_test_normalized[i, :].reshape(1,-1) - y_new)**2)
        #print(mse)
    return mse / X_test_normalized.shape[0]

In [ ]:
%%time
mse_big = calc_mse(X_test_normalized, X_train_normalized, y_train_normalized)

/usr/local/lib/python3.5/dist-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


## Step3.2: Get the small data and calculate mse 

In [7]:
small = sio.loadmat(datapath_small)
print(small['D'].shape)
pd.DataFrame(small['D']).head()

(83888, 10)


,0,1,2,3,4,5,6,7,8,9
0,0.000215,0.118160,-0.005951,16.030001,1.0,-1.0,0.000233,0.118159,-0.005951,16.030001
1,0.000233,0.118159,-0.005951,16.030001,1.0,-1.0,0.000257,0.118159,-0.005951,16.060001
2,0.000257,0.118159,-0.005951,16.060001,1.0,-1.0,0.000295,0.118159,-0.005951,16.060001
3,0.000295,0.118159,-0.005951,16.060001,1.0,-1.0,0.000334,0.118158,-0.005951,16.090002
4,0.000334,0.118158,-0.005951,16.090002,1.0,-1.0,0.000384,0.118158,-0.005951,16.090002


In [8]:
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(small['D'][:,:6], small['D'][:,6:], test_size=0.1, random_state=42)

In [12]:
min_param_small, max_param_small = compute_normalization_parameters(X_train_small)
min_param_y_small, max_param_y_small = compute_normalization_parameters(y_train_small)

In [13]:
X_train_normalized_small = normalize_data_per_row(X_train_small, min_param_small, max_param_small)
X_test_normalized_small = normalize_data_per_row(X_test_small, min_param_small, max_param_small)
y_train_normalized_small = normalize_data_per_row(y_train_small, min_param_y_small, max_param_y_small)
y_test_normalized_small = normalize_data_per_row(y_test_small, min_param_y_small, max_param_y_small)

In [ ]:
%%time
mse_big = calc_mse(X_test_normalized_small, X_train_normalized, y_train_normalized)

In [15]:
X_test_normalized_small[:100,:].shape

(100, 6)